# Analyse et Nettoyage du dataset UE

Seuls les véhicules des années 2022-2023 et de la France ont été exportés à partir de https://www.eea.europa.eu/data-and-maps/data/co2-cars-emission-20

# <font color='#3585CD'>Importation des librairies</font>

In [ ]:
import warnings
warnings.filterwarnings('ignore')
warnings.warn('DelftStack')
warnings.warn('Do not show this message')

import pandas as pd

import numpy as np

import matplotlib.pyplot as plt

import seaborn as sns

import plotly.figure_factory as ff
import plotly.express as px
import plotly.graph_objects as go

import kagglehub

from scipy.stats import gaussian_kde

import sys
import os

# <font color='#3585CD'>Fonctions qui seront utilisées dans ce notebook</font>

In [ ]:
def analyser_variable_categorielle_plotly(df, variable, top_n=100, display_array=True):
  """
  Analyse une variable catégorielle en affichant un DataFrame des 'top_n' catégories les plus fréquentes, ainsi qu'un graphique.

  :param df: DataFrame contenant la variable à analyser.
  :param variable: Nom de la variable catégorielle.
  :param top_n: Nombre de catégories à afficher (par défaut 100).
  """

  if display_array == True:
    top_cat = f"(Top {top_n} catégories)" if top_n != 100 else ""
    print(f"\n Analyse de la variable : {variable} {top_cat}")

  # Calcul des fréquences et pourcentages
  category_counts = df[variable].value_counts().head(top_n)
  category_percent = df[variable].value_counts(normalize=True).head(top_n) * 100

  # Création d’un DataFrame avec Libellé, Total et Pourcentage
  df_summary = pd.DataFrame({
      "Libellé": category_counts.index,
      "Total": category_counts.values,
      "Pourcentage": category_percent.values
  })

  # Affichage du tableau de synthèse
  display(df_summary)

  # Création du graphique interactif avec Plotly
  nb = top_n
  if top_n == 100:
    nb = ""
  fig = px.bar(df_summary,
                x="Libellé",
                y="Total",
                text="Total",
                title=f'Distribution des {nb} premières catégories de {variable}',
                labels={"Libellé": variable, "Total": "Nombre d'occurrences"},
                template="plotly_white")

  fig.update_traces(textposition='outside')
  fig.update_layout(xaxis_tickangle=-45)

  # Affichage du graphique
  fig.show()

def analyser_variables_numeriques_plotly(df, variables, bins=30):
    """
    Analyse les variables numériques en affichant un histogramme + KDE (distribution) et un boxplot interactifs.

    :param df: DataFrame contenant les données.
    :param variables: Liste des variables numériques à analyser.
    :param bins: Nombre de bins pour l'histogramme (par défaut 30).
    """
    for var in variables:
        #print(f"\n Analyse de la variable : {var}")
        #display(df[var].describe())  # Affichage des statistiques descriptives

        # Supprimer les valeurs NaN
        data = df[var].dropna()

        # Histogramme
        hist = go.Histogram(
            x=data,
            nbinsx=bins,
            marker=dict(color='skyblue', line=dict(color='black', width=1)),  # Bordures noires
            opacity=0.6,  # Semi-transparent pour voir la KDE
            name="Histogramme"
        )

        # Calcul des densités pour la courbe KDE
        kde = gaussian_kde(data)
        x_vals = np.linspace(data.min(), data.max(), 500)  # Intervalle lissé
        kde_vals = kde(x_vals)

        # Courbe KDE
        kde_curve = go.Scatter(
            x=x_vals,
            y=kde_vals * len(data) * (data.max() - data.min()) / bins,  # Mise à l'échelle par rapport à l'histogramme
            mode='lines',
            line=dict(color='blue', width=2),
            name="Densité (KDE)"
        )

        # Création de la figure combinée
        fig = go.Figure(data=[hist, kde_curve])

        # Mise en page
        fig.update_layout(
            title=f'Distribution de {var} (Histogramme + KDE)',
            xaxis_title="Valeur",
            yaxis_title="Fréquence",
            template="plotly_white",
            barmode='overlay'
        )
        fig.show()
        # # Affichage du graphique combiné
        # fig.show()

        # # Création du boxplot
        # boxplot = go.Box(
        #     x=data,
        #     marker=dict(color='salmon'),
        #     name="Boxplot",
        #     boxpoints="outliers"  # Affichage des outliers
        # )

        # # Création et affichage du Boxplot
        # fig_box = go.Figure(data=[boxplot])
        # fig_box.update_layout(
        #     title=f'Boxplot de {var}',
        #     xaxis_title="Valeur",
        #     template="plotly_white"
        # )

        # fig_box.show()


def display_missing_values(df):
  """
  Affiche un DataFrame contenant les valeurs manquantes, leur pourcentage / nombre et leur type, pour chaque colonne du DataFrame.

  :param df: DataFrame contenant la variable à analyser.
  """
  missing_values = df.isnull().sum()
  missing_ratio = (missing_values / len(df)) * 100

  missing_values_df = pd.DataFrame({
      'Colonne': missing_values.index,
      'Valeurs manquantes (%)': missing_ratio.values,
      'Nombre de valeurs manquantes': missing_values.values,
      'Type': df.dtypes.values
  })

  missing_values_df = missing_values_df[missing_values_df['Nombre de valeurs manquantes'] > 0]
  missing_values_df = missing_values_df.sort_values(by='Valeurs manquantes (%)', ascending=False).reset_index(drop=True)

  return missing_values_df

def afficher_boxplot(df, colonne, couleur="blue"):
  """
  Affiche un boxplot interactif pour une variable donnée.

  :param df: DataFrame contenant les données.
  :param colonne: Nom de la colonne à analyser.
  :param couleur: Couleur du boxplot (par défaut 'blue').
  """

  # Création de la figure
  fig = go.Figure()

  # Ajout du boxplot
  fig.add_trace(go.Box(
      x=df[colonne].dropna(),
      name=colonne,
      marker_color=couleur
  ))

  # Mise en page
  fig.update_layout(
      title=f"Box Plot des valeurs de {colonne}",
      xaxis_title=colonne,
      showlegend=False
  )

  # Affichage du graphique
  fig.show()

def plot_correlation_matrix(df):
  """
  Affiche la matrice de corrélation des variables numériques sous forme de heatmap interactive avec Plotly.

  :param df: DataFrame Pandas contenant les données
  """
  # Sélection des colonnes numériques
  num_numeric_cols = df.select_dtypes(include=['number']).columns

  # Calcul de la matrice de corrélation
  corr_matrix = df[num_numeric_cols].corr()

  # Création de la heatmap avec Plotly (labels en bas et à gauche)
  fig = ff.create_annotated_heatmap(
      z=corr_matrix.values,
      x=list(corr_matrix.columns),
      y=list(corr_matrix.index),
      colorscale="RdBu_r",
      annotation_text=corr_matrix.round(2).values,
      showscale=True
  )

  # Ajustement de la disposition
  fig.update_layout(
      title="Matrice de corrélation des variables numériques",
      height=600, width=800,
      xaxis=dict(side="bottom", tickangle=-45),
      yaxis=dict(side="left")
  )

  # Affichage
  fig.show()

def calculer_correlation(df, col1, col2):
  """
  Calcule et affiche la corrélation entre deux colonnes d'un DataFrame.

  Paramètres :
  df :DataFrame contenant les données.
  col1 : nom de la première colonne.
  col2 : nom de la deuxième colonne.

  Retourne la valeur de la corrélation et une interprétation de son intensité.
  """
  correlation = df[col1].corr(df[col2])
  print(f"Corrélation entre {col1} et {col2} : {correlation:.2f}")

  if abs(correlation) > 0.8:
      interpretation = "Très forte corrélation"
  elif abs(correlation) > 0.6:
      interpretation = "Forte corrélation"
  elif abs(correlation) > 0.4:
      interpretation = "Corrélation modérée"
  elif abs(correlation) > 0.15:
      interpretation = "Corrélation faible"
  else:
      interpretation = "Pas de corrélation linéaire significative"

  print(interpretation + ".\n")

  return correlation, interpretation

def detecter_outliers_plotly(df, seuil=1.5):
  """
  Détecte les outliers pour chaque variable numérique d'un DataFrame en utilisant la méthode IQR.
  Affiche également un Boxplot interactif pour chaque variable.

  :param df: DataFrame contenant les données.
  :param seuil: Seuil du coefficient IQR (par défaut 1.5).
  :return: DataFrame contenant le nombre d'outliers et le pourcentage par variable.
  """
  outliers_dict = {}

  for col in df.select_dtypes(include=['number']).columns:  # Sélectionner les colonnes numériques
      Q1 = df[col].quantile(0.25)  # Premier quartile
      Q3 = df[col].quantile(0.75)  # Troisième quartile
      IQR = Q3 - Q1  # Calcul de l'intervalle interquartile

      # Détection des valeurs aberrantes
      lower_bound = Q1 - seuil * IQR
      upper_bound = Q3 + seuil * IQR
      outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]

      # Stocker les résultats
      nb_outliers = outliers.shape[0]
      perc_outliers = (nb_outliers / df.shape[0]) * 100
      outliers_dict[col] = {"Nb_Outliers": nb_outliers, "Pourcentage": round(perc_outliers, 2)}

      # Création du Boxplot avec Plotly
      fig = go.Figure()
      fig.add_trace(go.Box(
          x=df[col],
          name=col,
          marker_color='blue',
          boxpoints='outliers'
      ))

      # Mise en page
      fig.update_layout(
          title=f"Box Plot de {col}",
          xaxis_title="Valeur",
          yaxis_title="Variable",
          template="plotly_white",
          showlegend=False
      )

      fig.show()

  # Conversion en DataFrame
  df_outliers = pd.DataFrame.from_dict(outliers_dict, orient='index')

  return df_outliers

def analyse_columns(df):

    """
    Analyse les colonnes d'un DataFrame : nom, type, nombre de valeurs uniques et exemples de valeurs.

    :param df: DataFrame Pandas
    :return: DataFrame avec l'analyse des colonnes
    """
    analysis = []

    for col in df.columns:
        col_name = col
        col_type = df[col].dtype
        unique_count = df[col].nunique()  # Nombre de valeurs uniques
        unique_values = df[col].dropna().unique()[:5].tolist()  # Exemples (max 5)
        unique_values = " | ".join(map(str, unique_values))  # Séparateur : " | "

        analysis.append({
            'Nom de la colonne': col_name,
            'Type de la colonne': col_type,
            'Nombre de valeurs uniques': unique_count,
            'Exemples de valeurs': unique_values
        })

    return pd.DataFrame(analysis)

def plot_scatter_co2(df, x, y="CO2", color="Carburant", size="CO2"):
  """
  Génère un scatter plot interactif avec Plotly, avec une ligne de moyenne CO2.

  Paramètres :
  - df : DataFrame contenant les données
  - x : Nom de la colonne pour l'axe X
  - y : Nom de la colonne pour l'axe Y (par défaut "CO2")
  - color : Nom de la colonne pour la couleur des points (par défaut "Carburant")
  - size : Nom de la colonne pour la taille des points (par défaut "CO2")
  """

  # Calcul de la moyenne globale du CO2
  moyenne_co2 = df[y].mean()

  # Création du scatter plot
  fig = px.scatter(
      df,
      x=x,
      y=y,
      color=color,
      size=size,
      title=f"Relation entre {x} et {y}",
      labels={x: x.capitalize(), y: y.capitalize(), color: color.capitalize()},
      hover_data=df.columns,
      size_max=20
  )

  # Ajout de la ligne de moyenne CO2
  fig.add_hline(
      y=moyenne_co2,
      line_dash="dot",
      line_color="red",
      annotation_text=f"Moyenne CO2: {moyenne_co2:.2f} g/km",
      annotation_position="top right",
      annotation_font_color="red",
      annotation_font_size=12,
      annotation_bgcolor="rgba(255,255,255,0.7)"
  )

  fig.show()


def analyser_hist_co2_par_variable(df, variable, top_n=50, order='desc', co2="CO2"):
  """
  Génère un histogramme interactif avec Plotly, avec une ligne de moyenne CO2.

  Paramètres :
  :param df : DataFrame contenant les données
  :param variable : Nom de la colonne à analyser
  :param top_n: Nombre de catégories à afficher (par défaut 50).
  :param order: Ordonnancement des catégories (par défaut 'desc').
  :param co2: Nom de la colonne des émissions de CO2 (par défaut "CO2")
  """
  ascending = True if order == 'asc' else False

  df_co2 = df.groupby(variable)[co2].mean().sort_values(ascending=ascending).reset_index()
  df_co2 = df_co2.head(top_n)

  df_co2['CO2_txt'] = df_co2[co2].apply(lambda x: f"{x:.2f}")

  moyenne_co2 = df[co2].mean()

  fig = px.bar(
      df_co2,
      x=variable,
      y=co2,
      title=f"Distribution des émissions de CO2 par {variable} (Top {top_n})",
      labels={variable: variable.capitalize(), co2: "Émissions de CO2 (g/km)"},
      color=co2,
      color_continuous_scale="RdYlGn_r",
      text='CO2_txt'
  )

  # Ligne de moyenne du CO2
  fig.add_hline(
      y=moyenne_co2,
      line_dash="dot",
      line_color="red",
      annotation_text=f"Moyenne CO2: {moyenne_co2:.2f} g/km",
      annotation_position="top right",
      annotation_font_color="red",
      annotation_font_size=12,
      annotation_bgcolor="rgba(255,255,255,0.7)"
  )

  fig.update_layout(
      xaxis_tickangle=-75,
      coloraxis_colorbar=dict(title="CO2 (g/km)"),
      uniformtext_minsize=8,
      uniformtext_mode='hide'
  )

  fig.show()

def detecter_outliers_plotly_var(serie, seuil=1.5):
    """
    Détecte les outliers pour une seule variable numérique en utilisant la méthode IQR.
    Affiche également un Boxplot interactif.

    :param serie: Série Pandas contenant les valeurs de la variable.
    :param seuil: Seuil du coefficient IQR (par défaut 1.5).
    :return: DataFrame contenant le nombre d'outliers et le pourcentage.
    """
    if not isinstance(serie, pd.Series):
        raise ValueError("Veuillez fournir une variable sous forme de pd.Series")

    Q1 = serie.quantile(0.25)  # Premier quartile
    Q3 = serie.quantile(0.75)  # Troisième quartile
    IQR = Q3 - Q1  # Intervalle interquartile

    # Détection des valeurs aberrantes
    lower_bound = Q1 - seuil * IQR
    upper_bound = Q3 + seuil * IQR
    outliers = serie[(serie < lower_bound) | (serie > upper_bound)]

    # Résultats
    nb_outliers = outliers.shape[0]
    perc_outliers = (nb_outliers / serie.shape[0]) * 100

    # Création du Boxplot avec Plotly
    fig = go.Figure()
    fig.add_trace(go.Box(
        x=serie,
        name=serie.name if serie.name else "Variable",
        marker_color='blue',
        boxpoints='outliers'
    ))

    # Mise en page
    fig.update_layout(
        title=f"Box Plot de {serie.name if serie.name else 'Variable'}",
        xaxis_title="Valeur",
        yaxis_title="Variable",
        template="plotly_white",
        showlegend=False
    )

    fig.show()

    # Résultats sous forme de DataFrame
    df_outliers = pd.DataFrame({
        "Nb_Outliers": [nb_outliers],
        "Pourcentage": [round(perc_outliers, 2)]
    }, index=[serie.name if serie.name else "Variable"])

    return df_outliers


# <font color='#3585CD'>Chargement des données</font>

## Chargement du dataset principal

In [ ]:
# Le dataset étant volumineux pour être dans le repository Github, nous le téléchargeons via Kaggle test
path = kagglehub.dataset_download("dimitrileloup/vehicules-fr-2022-2023")
print("Chemin vers le fichier : ", path)

In [ ]:
dataset_path = f"{path}/datas_FR_2022_2023.csv"
df = pd.read_csv(dataset_path)
df.head(10)

In [ ]:
# Suppression des espaces accidentels dans les noms des colonnes comme "Fuel consumption "
df.columns = df.columns.str.strip()

## Chargement du dataset du dictionnaire des variables

In [ ]:
path_vars = kagglehub.dataset_download("dimitrileloup/dfinition-des-colonnes")
print("Path to dataset files:", path_vars)

In [ ]:
pd.set_option('display.max_colwidth', None) # pour pouvoir afficher tout le descriptif
dataset_variables = f"{path_vars}/Table-definition.xlsx"
var = pd.read_excel(dataset_variables)
var.head(40)

# <font color='#3585CD'>Premières analyses</font>

## Informations sur le dataset

In [ ]:
print("\nAperçu du dataset :")
print(df.info())

## Satistiques descriptives

In [ ]:
print("\nStatistiques descriptives :")
display(df.describe(include='number').T)

In [ ]:
df.describe(include="object").T

## Nombre de valeurs uniques par colonne

In [ ]:
# Calculer le nombre de valeurs uniques par colonne et renommer la colonne
df_unique_values = df.nunique().sort_values(ascending=False).reset_index()
df_unique_values.columns = ["colonne", "nombre de valeurs uniques"]

df_unique_values

## Analyse rapide des colonnes
<p>La fonction analyse_columns permet de visualiser les différentes colonnes avec un échantillon de valeurs</p>

In [ ]:
analysis_df = analyse_columns(df)
analysis_df

# <font color='#3585CD'>Analyse des valeurs manquantes</font>

In [ ]:
data_na = display_missing_values(df)
data_na

# <font color='#3585CD'>Suppression de colonnes</font>

Nous pouvons dès à présent ces colonnes qui ont un taux de valeurs manquantes supérieur à 70% :

*   At2 (mm)
*   W (mm)
*   MMS
*   Vf
*   De
*   Ernedc (g/km)
*   At1 (mm)
*   Enedc (g/km)
*   RLFI
*   z (Wh/km)
*   Electric range (km)

In [ ]:
df = df.drop(columns=['At2 (mm)', 'W (mm)', 'MMS', 'Vf', 'De', 'Ernedc (g/km)',	'At1 (mm)',	'Enedc (g/km)',	'RLFI',	'z (Wh/km)', 'Electric range (km)'], axis=1)

In [ ]:
df.duplicated().sum()

In [ ]:
data_na = display_missing_values(df)
data_na

# <font color='#3585CD'>Suppresion des colonnes non pertinentes</font>

Certaines colonnes n'ont pas d'intérêt à être gardées :

*   IT
*   Erwltp (g/km) (dépreciée)
*   ID : identifiant du véhicule
*   Country : notre dataset est une extraction des véhicules de France
*   VFN : n'a pas de norme universelle et comporte trop de valeurs
*   Tan : trop de valeurs et sans intérêt pour notre projet
*   T : trop de valeurs et sans intérêt pour notre projet
*   Va : trop de valeurs et sans intérêt pour notre projet
*   Ve : trop de valeurs et sans intérêt pour notre projet
*   Status : n'a qu'une seule valeur et ne varie pas
*   Year : 1 seule valeur
*   Date of registration : sans intérêt pour notre projet
*   Fm : redondant avec Ft
*   Cr : nous avons 2 catégories (M1, M1G). M1G est une sous-catégorie de M1 réservée aux véhicules tout-terrain. Nous pouvons conclure que tous les véhicules sont de catégorie M1
*   Ct : idem que Cr
*   ech : sans intérêt pour notre projet
*   Mp : redondant, se retrouve dans une autre colonne
*   Man : redondant avec Mk
*   r : n'a qu'une seule valeur
*   Mh : redondant avec Mk


In [ ]:
df = df.drop(columns=['IT', 'Erwltp (g/km)', 'ID', 'Country', 'VFN', 'Tan', 'T', 'Va', 'Ve', 'Status', 'year', 'Date of registration', 'Fm', 'Cr', 'Ct', 'ech', 'Mp', 'Man', 'r', 'Mh'], axis=1)

## Vérification de la corrélation entre Masse à vide et Masse totale</font>

In [ ]:
df_masse = df[['Mt', 'm (kg)']]
plot_correlation_matrix(df_masse)

On se rend compte qu'il y a une **forte corrélataion** entre ces 2 variables, qui pourrait entrainer une **colinéarité**. Nous prenons la décision de ne garder que la masse à vide (m (kg))

In [ ]:
df = df.drop('Mt', axis=1)

# <font color='#3585CD'>Gestion des doublons</font>

## Nombre de doublons

In [ ]:
df.duplicated().sum()

## Supression des doublons

In [ ]:
df.drop_duplicates(inplace =True)

In [ ]:
df.duplicated().sum()

In [ ]:
df.shape

In [ ]:
df = df.reset_index(drop=True)

# <font color='#3585CD'>Renommage des colonnes</font>

Pour plus de compréhension, nous allons renommer les colonnes :



*   Mk : Marque
*   Cn : Modele
*   Ewltp (g/km) : Co2
*   Ft : Carburant
*   ec (cm3) : Cylindree moteur
*   ep (KW) : Puissance moteur
*   Fuel consumption : Consommation carburant


In [ ]:
renommage = {
    'Mk': 'Marque',
    'Cn': 'Modèle',
    'm (kg)' : 'Masse à vide',
    'Ewltp (g/km)': 'CO2',
    'Ft': 'Carburant',
    'ec (cm3)': 'Cylindrée moteur',
    'ep (KW)': 'Puissance moteur',
    'Fuel consumption': 'Consommation carburant'
}

# Application du renommage
df.rename(columns=renommage, inplace=True)

In [ ]:
df

# <font color='#3585CD'>Faut-il garder les véhicules électriques et hydrogènes ?</font>
Notre objectif est de prédire les émissions directes de CO2. Garder les véhicules électriques et hydrogènes risque de biaiser notre modèle. Nous allons donc exclure les véhicules électriques de notre dataset

In [ ]:
# Vérification des émissions de CO2 des véhicules hydrogènes
df_hydrogen = df[df["Carburant"] == "hydrogen"]
df_hydrogen['CO2'].value_counts()

In [ ]:
# Nous excluons les véhicules électriques et hydrogènes
df = df[(df["Carburant"] != "electric") & (df["Carburant"] != "hydrogen")]

In [ ]:
df.shape

# <font color='#3585CD'>Traitement des valeurs manquantes</font>

In [ ]:
data_na = display_missing_values(df)
data_na

## Gestion des valeurs manquantes de la colonne Consommation carburant

In [ ]:
df_fc_na = df[df['Consommation carburant'].isna()]
df_fc_na

Nous allons gérer cette ligne

In [ ]:
# regardons si d'autres modeles RANGE ROVER EVOQUE sont renseignés
df_jag_evoque = df[(df['Modèle'] == 'RANGE ROVER EVOQUE') & (df['Carburant'] == 'diesel') & (~df['Consommation carburant'].isna())]
df_jag_evoque.head()

In [ ]:
fc_mean_jag_evoque = df_jag_evoque['Consommation carburant'].mean().round(1)
fc_mean_jag_evoque

In [ ]:
df.loc[(df["Modèle"] == "RANGE ROVER EVOQUE") & (df['Consommation carburant'].isna()), "Consommation carburant"] = fc_mean_jag_evoque

In [ ]:
data_na = display_missing_values(df)
data_na

## Gestion des valeurs manquantes de la colonne Masse à vide

In [ ]:
data_na = display_missing_values(df)
data_na

In [ ]:
df_mkg_na = df[df['Masse à vide'].isna()]
df_mkg_na

Recherchons dans le dataset si nous avons des modèles équivalents dont les variables m (kg) et Mt ne sont **pas nulles**

In [ ]:
df_jag_lr = df[(df['Modèle'] == 'RANGE ROVER EVOQUE') & (df['Carburant'] == 'diesel') & (df['Puissance moteur'] == 120) & (~df['Masse à vide'].isna())]
df_jag_lr

In [ ]:
fc_mean_masse_jag_evoque = df_jag_evoque['Masse à vide'].mean().round(1)
fc_mean_masse_jag_evoque

In [ ]:
# pour les mêmes modèles, la variable Masse à vide est égale à 1967
df['Masse à vide'] = df['Masse à vide'].fillna(fc_mean_masse_jag_evoque)

In [ ]:
# Vérification
df.loc[23942].to_frame().T

In [ ]:
data_na = display_missing_values(df)
data_na

In [ ]:
# ré inisialisation des indexes
df = df.reset_index(drop=True)
df

# <font color='#3585CD'>Distribution des variables catégorielles</font>

## Analyse par marque

### Valeurs uniques

In [ ]:
sorted(df['Marque'].unique())

### Remplacement

Certaines valeurs peuvent être regroupées, comme par exemple :

*   'MC LAREN', 'MCLAREN'
*   'MERCEDES AMG', 'MERCEDES BENZ', 'MERCEDES-BENZ'
*   'MITSUBISHI', 'MITSUBISHI MOTORS CORPORATION', 'MITSUBISHI MOTORS THAILAND'


In [ ]:
replace_mk = {'MC LAREN' : 'MCLAREN',
              'MERCEDES AMG' : 'MERCEDES BENZ',
              'MERCEDES-BENZ' : 'MERCEDES BENZ',
              'MITSUBISHI MOTORS CORPORATION' : 'MITSUBISHI',
              'MITSUBISHI MOTORS THAILAND' : 'MITSUBISHI',
              'MITSUBISHI MOTORS (THAILAND)' : 'MITSUBISHI',
              'FORD-CNG-TECHNIK' : 'FORD',
              'ROLLS ROYCE' : 'ROLLS-ROYCE'}
df['Marque'] = df['Marque'].replace(replace_mk)

In [ ]:
df['Marque'].nunique()

### Analyse

In [ ]:
analyser_variable_categorielle_plotly(df, 'Marque', 58)

## Analyse par carburant

### Valeurs uniques

In [ ]:
sorted(df['Carburant'].unique())

### Remplacement

Certaines valeurs peuvent être regroupées :



*   'diesel/electric' & 'petrol/electric' sont des véhicules hybride
*   'lpg' & 'ng' sont des énergies au gaz
*   'petrol' sera renommé en 'essence' pour plus de compréhension.


In [ ]:
replace_ft = {
              'petrol' : 'essence',
              'diesel/electric' : 'hybride',
              'petrol/electric' : 'hybride',
              'lpg' : 'gaz',
              'ng' : 'gaz'}
df['Carburant'] = df['Carburant'].replace(replace_ft)

### Analyse

In [ ]:
analyser_variable_categorielle_plotly(df, 'Carburant')

## Analyse par modèle de voiture

In [ ]:
analyser_variable_categorielle_plotly(df, 'Modèle', 30)

# <font color='#3585CD'>Distribution des variables numériques</font>

## Sélection des colonnes numériques

In [ ]:
# Sélection des colonnes numériques
num_vars = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
num_vars

## Analyse de la masse du véhicule Masse à vide

In [ ]:
analyser_variables_numeriques_plotly(df, ['Masse à vide'])

## Analyse des émissions spécifiques de CO2

In [ ]:
analyser_variables_numeriques_plotly(df, ['CO2'], 30)

## Analyse de la cylindrée moteur

In [ ]:
analyser_variables_numeriques_plotly(df, ['Cylindrée moteur'], 50)

## Analyse de la puissance du moteur

In [ ]:
analyser_variables_numeriques_plotly(df, ['Puissance moteur'])

## Analyse de la Consommation carburant

In [ ]:
analyser_variables_numeriques_plotly(df, ['Consommation carburant'])

# <font color='#3585CD'>Création d'indicateurs</font>

## Indicateur de Charge Spécifique du Moteur (ICSM)

`ICSM = Puissance (kW) / Masse du véhicule (kg)`

**Interprétation** :

*   Faible ICSM → Voiture puissante et légère (moins d’effort, moins de CO₂).
*   Élevé ICSM → Voiture sous-motorisée (forte sollicitation, plus de CO₂).

In [ ]:
df['ICSM'] = df['Puissance moteur'] / df['Masse à vide']

## Indicateur de Consommation Énergétique (ICE)

`ICE = Puissance (kW) / Cylindrée (cm³)`

**Interprétation** :

*   Faible ICE → Moteur optimisé (ex. turbo downsizing).
*   Élevé ICE → Moteur gourmand et peu efficient.

In [ ]:
df['ICE'] = df['Puissance moteur'] / df['Cylindrée moteur']

## Indicateur de Densité Energétique du Carburant (IDEC)

`IDEC = Cylindrée (cm³) / Masse du véhicule (kg)`

**Interprétation** :

*   Faible IDEC → Moteur bien dimensionné (moins d’effort, moins de CO₂).
*   Élevé IDEC → Moteur sous-dimensionné (forte sollicitation, plus de CO₂).

In [ ]:
df['IDEC'] = df['Cylindrée moteur'] / df['Masse à vide']

## Indicateur de Consommation Spécifique (ICS)

`ICS = Puissance produite (kWh) / Consommation de carburant (g)`

**Interprétation** :

*   Faible ICS : Moteur efficace, consomme moins de carburant et génère moins de CO₂ pour produire la même puissance.
*   Élevé ICS : Moteur inefficace, consomme plus de carburant et génère plus de CO₂ pour produire la même puissance.

In [ ]:
df['ICS'] = df['Puissance moteur'] / df['Consommation carburant']

In [ ]:
df

# <font color='#3585CD'>Analyse du CO2 en fonction de certaines variables</font>

### Analyse du CO2 en fonction de la masse du véhicule

In [ ]:
plot_scatter_co2(df, "Masse à vide")

On observe que les véhicules plus lourds tendent à émettre plus de CO₂, avec une distinction entre les types de carburant.

### Analyse du CO2 en fonction de la puissance du moteur.

In [ ]:
plot_scatter_co2(df, "Puissance moteur")

### Analyse du CO2 en fonction du carburant



In [ ]:
plot_scatter_co2(df, "Carburant")

In [ ]:
analyser_hist_co2_par_variable(df, 'Carburant')

### Analyse du CO2 en fonction de la marque

**Top des marques les plus polluantes**

In [ ]:
analyser_hist_co2_par_variable(df, 'Marque', 36)

### Analyse du CO2 en fonction du modèle de voiture

**Top des modèles les plus polluants**

In [ ]:
analyser_hist_co2_par_variable(df, 'Modèle', 30)

**Top des modèles les moins polluants**

In [ ]:
analyser_hist_co2_par_variable(df, 'Modèle', 30, order='asc')

### Analyse du CO2 en fonction de la consommation

In [ ]:
plot_scatter_co2(df, "Consommation carburant")

### Analyse du CO2 en fonction des indicateurs ICSM, ICE, IDEC, ICS

In [ ]:
plot_scatter_co2(df, "ICSM")

In [ ]:
plot_scatter_co2(df, "ICE")

In [ ]:
plot_scatter_co2(df, "IDEC")

In [ ]:
plot_scatter_co2(df, "ICS")

# <font color='#3585CD'>Corrélation des variables numériques</font>

In [ ]:
plot_correlation_matrix(df)

In [ ]:
num_numeric_cols = df.select_dtypes(include=['number']).columns
num_numeric_cols

for col in num_numeric_cols:
  calculer_correlation(df, col, 'CO2')

# <font color='#3585CD'>Analyse des outliers</font>

## Masse à vide

In [ ]:
detecter_outliers_plotly_var(df['Masse à vide'])

In [ ]:
df_outliers_masse = df[df['Masse à vide'] < 750].sort_values(by='Masse à vide', ascending=True)
df_outliers_masse.head(20)

In [ ]:
df_outliers_masse = df[df['Masse à vide'] > 2500].sort_values(by='Masse à vide', ascending=True)
df_outliers_masse.tail(20)

## Cylindrée moteur

In [ ]:
detecter_outliers_plotly_var(df['Cylindrée moteur'])

In [ ]:
df_outliers_CO2 = df[df['Cylindrée moteur'] > 4000].sort_values(by='Cylindrée moteur', ascending=True)
df_outliers_CO2.tail(20)

## CO2

In [ ]:
detecter_outliers_plotly_var(df['CO2'])

In [ ]:
df_outliers_CO2 = df[df['CO2'] > 400].sort_values(by='CO2', ascending=True)
df_outliers_CO2.tail(20)

## Puissance moteur

In [ ]:
detecter_outliers_plotly_var(df['Puissance moteur'])

In [ ]:
df_outliers_Consommation = df[df['Puissance moteur'] > 15].sort_values(by='Puissance moteur', ascending=True)
df_outliers_Consommation.tail(20)

## Consommation carburant

In [ ]:
detecter_outliers_plotly_var(df['Consommation carburant'])

In [ ]:
df_outliers_Consommation = df[df['Consommation carburant'] > 15].sort_values(by='Consommation carburant', ascending=True)
df_outliers_Consommation.tail(20)

# <font color='#3585CD'>Visualisation globale graphique</font>

In [ ]:
fig = px.scatter_matrix(df, dimensions=df.select_dtypes(include=['number']).columns,
                        color='Carburant', title="Pairplot")

fig.update_layout(height=900, width=1200)
fig.show()

In [ ]:
plot_correlation_matrix(df)

# <font color='#3585CD'>Distribution de la variable cible</font>





## Histogramme et boxplot de la variable cible

In [ ]:
analyser_variables_numeriques_plotly(df, ['CO2'])

In [ ]:
import plotly.graph_objects as go
import numpy as np
import scipy.stats as stats
import pandas as pd

# Calcul des quantiles
(quantiles, values), (slope, intercept, r) = stats.probplot(df['CO2'], dist="norm")

# Création du Q-Q Plot avec Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=quantiles, y=values, mode='markers', name='Données'))
fig.add_trace(go.Scatter(x=quantiles, y=slope * np.array(quantiles) + intercept, mode='lines', name='Ligne théorique'))

fig.update_layout(title='Q-Q plot pour CO2', xaxis_title='Quantiles théoriques', yaxis_title='Quantiles des données')

fig.show()

In [ ]:
from statsmodels.stats.diagnostic import lilliefors
# Niveau de signification (alpha)
alpha = 0.05

# Effectuer les tests de normalité
shapiro_test = stats.shapiro(df['CO2'])
ks_test = stats.kstest(df['CO2'], 'norm')
ad_test = stats.anderson(df['CO2'], dist='norm')
dagostino_test = stats.normaltest(df['CO2'])
lilliefors_test = lilliefors(df['CO2'], dist='norm')

# Créer un tableau pandas avec les résultats des tests
test_results = pd.DataFrame({
    'Nom du test': ['Shapiro-Wilk', 'Kolmogorov-Smirnov', 'Anderson-Darling', "D'Agostino-Pearson", 'Lilliefors'],
    'Statistique de test': [shapiro_test[0], ks_test.statistic, ad_test.statistic, dagostino_test.statistic, lilliefors_test[0]],
    'p-valeur': [shapiro_test[1], ks_test.pvalue, None, dagostino_test.pvalue, lilliefors_test[1]],
    'Normalité': ['Oui' if shapiro_test[1] > alpha else 'Non',
                  'Oui' if ks_test.pvalue > alpha else 'Non',
                  'Oui' if any(ad_test.statistic < crit_val for crit_val in ad_test.critical_values) else 'Non',
                  'Oui' if dagostino_test.pvalue > alpha else 'Non',
                  'Oui' if lilliefors_test[1] > alpha else 'Non']
})

print(test_results)


In [ ]:
# from scipy.stats import boxcox

# df['co2_transformed'], lambda_boxcox = boxcox(df['Ewltp (g/km)'] + 1)  # Ajouter 1 pour éviter 0
# df

In [ ]:
df['Carburant'].unique()

In [ ]:
# Histogramme en fonction du type de motorisation
fig = px.histogram(df, x="CO2", color="Carburant", nbins=50, barmode="overlay",
                   title="Distribution des émissions de CO2 par type de carburant")
fig.show()


Comment gérer ?


*   Création colonne Hybride ?
*   Dataset à part ?
*   Juste un OneHotEncoder ?

## Degrés d'asymétrie des variables

In [ ]:
num_vars = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
df[num_vars].skew().sort_values()

### CO2

Légère asymétrie négative (peu inquiétante)

### Consommation carburant

 Quasi symétrique (très léger)

### Masse moyenne

Asymétrie positive modérée à forte, probablement due à des valeurs élevées qui tirent la distribution.

# <font color='#3585CD'>Quelles variables garder pour prédire le CO2 ?</font>

Selon nous, les **variables pertinentes** à garder pour la prédiction sont les suivantes :
* **Masse à vide** : un véhicule plus lourd a souvent des émissions plus élevées.
* **Cylindrée moteur** : une plus grande cylindrée est souvent associée à une plus grande consommation et donc plus d’émissions.
* **Puissance moteur** : un moteur plus puissant a tendance à consommer plus de carburant.
* **Carburant** : Essence, diesel, hybride… Chaque type influence les émissions. La variable Carburant sera encodée.

* **Consommation carburant** : directement liée aux émissions de CO2. Cependant sa forte corrélation peut expliquer à elle seule les émissions de CO2. Nous nous questionnerons au moment de la modélisation si nous la gardons ou pas.

Nous **excluerons** les variables **Marque** et **Modèle** qui ne sont pas directement liées aux émissions de CO2.

Pourquoi exclure les variables **Marque** et **Modèle** ?
* Ce sont des variables catégoriques à très haute **cardinalité**
* L'influence sur le CO₂ passe par des **caractéristiques techniques**

# <font color='#3585CD'>Export du dataset nettoyé</font>

In [ ]:
df.shape

In [ ]:
df = df.reset_index(drop=True)
df

In [ ]:
# Vérifier si on est sur Google Colab
import os
try:
    import google.colab
    ON_COLAB = True
    dataset_path = "/content/drive/My Drive/Formation DS/Projet CO2/NOV24-CDS-CO2/notebooks/datasets/Dataset_final/datas_nettoyees_model_FR.csv"

    # Monter Google Drive si ce n'est pas déjà fait
    from google.colab import drive
    drive.mount('/content/drive')

except ImportError:
    ON_COLAB = False
    dataset_path = "datasets/Dataset_final/datas_nettoyees_model_FR.csv"

# Sauvegarde du DataFrame
df.to_csv(dataset_path, index=False)

# Vérification de l'enregistrement
if os.path.exists(dataset_path):
    print(f"Le fichier a bien été enregistré à l'emplacement : {dataset_path}")
else:
    print("Problème lors de l'enregistrement du fichier.")


In [ ]:
# dataset_path = "datasets/Dataset_final/datas_nettoyees_model_FR.csv"
# df.to_csv(dataset_path, index =False)

# import os

# # Vérifie si le fichier existe
# if os.path.exists(dataset_path):
#     print("Le fichier a bien été enregistré.")
# else:
#     print("Problème lors de l'enregistrement du fichier.")
